In [1]:
import os

import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lars, Lasso, LassoLars, OrthogonalMatchingPursuit, ARDRegression, BayesianRidge
from sklearn.svm import LinearSVR, NuSVR, SVR
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import r2_score

import xgboost as xgb

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

In [2]:
aws_df = pd.read_excel("/Users/kpmac/Documents/IITB/DS593/data/AWS/rainfall-2015_2023.xlsx")
aws_df.head()

,Index2,Andheri,B ward,Bandra,Byculla,C ward,Chembur,Chincholi,Colaba,D Ward,...,Mulund,N ward,Nariman Fire,Rawali camp,S ward,SWD Workshop dadar,Thakare natya,Vikhroli,Worli,vileparle W
0,2015-06-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-06-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-06-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-06-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-06-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
aws_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105408 entries, 0 to 105407
Data columns (total 37 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Index2              105408 non-null  datetime64[ns]
 1   Andheri             105400 non-null  float64       
 2   B ward              105342 non-null  float64       
 3   Bandra              105388 non-null  float64       
 4   Byculla             105402 non-null  float64       
 5   C ward              105400 non-null  float64       
 6   Chembur             105386 non-null  float64       
 7   Chincholi           105153 non-null  float64       
 8   Colaba              105392 non-null  float64       
 9   D Ward              105402 non-null  float64       
 10  Dahisar             105394 non-null  float64       
 11  Dindoshi            105402 non-null  float64       
 12  F North             105391 non-null  float64       
 13  F South             105398 no

In [4]:
aws_df.describe()

,Index2,Andheri,B ward,Bandra,Byculla,C ward,Chembur,Chincholi,Colaba,D Ward,...,Mulund,N ward,Nariman Fire,Rawali camp,S ward,SWD Workshop dadar,Thakare natya,Vikhroli,Worli,vileparle W
count,105408,105400.000000,105342.000000,105388.000000,105402.000000,105400.000000,105386.000000,105153.000000,105392.000000,105402.000000,...,105388.000000,105397.000000,105394.000000,105401.000000,105142.000000,105360.000000,105393.000000,105389.000000,105377.000000,105379.000000
mean,2019-08-01 07:52:30,0.221814,0.189982,0.200552,0.198314,0.176791,0.226229,0.216861,0.185875,0.187949,...,0.228194,0.219225,0.167074,0.195001,0.244541,0.228224,0.235435,0.246020,0.207730,0.208006
min,2015-06-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2017-07-01 11:56:15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2019-07-31 23:52:30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2021-08-31 11:48:45,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2023-09-30 23:45:00,45.720000,28.450000,35.560000,27.690000,27.690000,35.560000,29.970000,28.700000,36.320000,...,53.340000,33.020000,25.400000,29.720000,32.770000,29.970000,34.290000,33.780000,33.780000,34.800000
std,NaN,1.097824,0.943621,1.012277,0.978841,0.884220,1.085415,1.041343,0.938328,0.955546,...,1.040737,1.013211,0.849970,0.950343,1.094797,1.097404,1.098261,1.135192,1.072614,1.043341


In [5]:
aws_df.isna().sum(axis=0)

Index2                  0
Andheri                 8
B ward                 66
Bandra                 20
Byculla                 6
C ward                  8
Chembur                22
Chincholi             255
Colaba                 16
D Ward                  6
Dahisar                14
Dindoshi                6
F North                17
F South                10
G South                34
Gowanpada              94
H West ward            95
K East ward            12
K West ward             6
Kandivali               5
Kurla                  10
L ward                 21
M West ward            13
MCGM 1                 88
Malvani                21
Marol                   5
Memonwada               9
Mulund                 20
N ward                 11
Nariman Fire           14
Rawali camp             7
S ward                266
SWD Workshop dadar     48
Thakare natya          15
Vikhroli               19
Worli                  31
vileparle W            29
dtype: int64

In [6]:
aws_df.fillna(0, inplace=True)

Some data is missing for forecasts at 00Z of PR in 2015. Thus dropping all the data corresponding to 2015.

Also, PR analysis data is only from 2019, so need a way to do some augmentation.

Ignoring PR analysis data for now since PW is available

In [7]:
aws_df.rename(columns={"Index2": "Time"}, inplace=True)

In [8]:
aws_df = aws_df[aws_df["Time"].dt.year.isin([year for year in range(2016, 2024)])].reset_index(drop=True)

14 July 2022 shows a lot of missing data based on filename analysis that was done at the end of the notebook, so remove the corresponding data points from aws_normalized (also remove 15 July since these points will require GFS forecasts from 14 July based on current model)

In [9]:
aws_df = aws_df[~aws_df["Time"].dt.date.isin([pd.to_datetime("2022-07-14").date(), pd.to_datetime("2022-07-15").date()])].reset_index(drop=True)

In [10]:
aws_df

,Time,Andheri,B ward,Bandra,Byculla,C ward,Chembur,Chincholi,Colaba,D Ward,...,Mulund,N ward,Nariman Fire,Rawali camp,S ward,SWD Workshop dadar,Thakare natya,Vikhroli,Worli,vileparle W
0,2016-06-01 00:00:00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00
1,2016-06-01 00:15:00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00
2,2016-06-01 00:30:00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00
3,2016-06-01 00:45:00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00
4,2016-06-01 01:00:00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93499,2023-09-30 22:45:00,0.51,0.2,0.76,0.25,0.51,0.51,0.51,0.25,0.51,...,0.25,0.6,0.25,0.51,0.51,0.51,0.25,0.4,0.6,0.76
93500,2023-09-30 23:00:00,0.51,0.4,0.51,0.51,0.25,0.25,0.51,0.76,0.25,...,0.25,0.2,0.51,0.25,0.00,0.25,0.51,0.4,0.4,0.51
93501,2023-09-30 23:15:00,0.25,0.2,0.00,0.25,0.25,0.25,0.00,0.00,0.25,...,0.00,0.2,0.00,0.25,0.25,0.25,0.25,0.0,0.2,0.00
93502,2023-09-30 23:30:00,0.00,0.0,0.00,0.00,0.00,0.00,0.25,0.25,0.00,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00


In [11]:
aws_df.iloc[5, :]

Time                  2016-06-01 01:15:00
Andheri                               0.0
B ward                                0.0
Bandra                                0.0
Byculla                               0.0
C ward                                0.0
Chembur                               0.0
Chincholi                             0.0
Colaba                                0.0
D Ward                                0.0
Dahisar                               0.0
Dindoshi                              0.0
F North                               0.0
F South                               0.0
G South                               0.0
Gowanpada                             0.0
H West ward                           0.0
K East ward                           0.0
K West ward                           0.0
Kandivali                             0.0
Kurla                                 0.0
L ward                                0.0
M West ward                           0.0
MCGM 1                            

In [12]:
regions = aws_df.columns[1:]
regions

Index(['Andheri', 'B ward', 'Bandra', 'Byculla', 'C ward', 'Chembur',
       'Chincholi', 'Colaba', 'D Ward', 'Dahisar', 'Dindoshi', 'F North',
       'F South', 'G South', 'Gowanpada', 'H West ward', 'K East ward',
       'K West ward', 'Kandivali', 'Kurla', 'L ward', 'M West ward', 'MCGM 1',
       'Malvani', 'Marol', 'Memonwada', 'Mulund', 'N ward', 'Nariman Fire',
       'Rawali camp', 'S ward', 'SWD Workshop dadar', 'Thakare natya',
       'Vikhroli', 'Worli', 'vileparle W'],
      dtype='object')

In [13]:
aws_df.shape

(93504, 37)

In [14]:
aws_array = aws_df.values[:, 1:]
print(aws_array.shape)

(93504, 36)


In [15]:
for j in range(len(regions)):
    print(regions[j])
    counts, bins = np.histogram(aws_array[:, j])
    for i in range(len(counts)):
        print(f"({bins[i]}, {bins[i+1]}): {counts[i]}")
    print()

Andheri
(0.0, 4.572): 92400
(4.572, 9.144): 796
(9.144, 13.716000000000001): 205
(13.716000000000001, 18.288): 59
(18.288, 22.86): 26
(22.86, 27.432000000000002): 12
(27.432000000000002, 32.004): 4
(32.004, 36.576): 1
(36.576, 41.148): 0
(41.148, 45.72): 1

B ward
(0.0, 2.8449999999999998): 91760
(2.8449999999999998, 5.6899999999999995): 1098
(5.6899999999999995, 8.535): 374
(8.535, 11.379999999999999): 149
(11.379999999999999, 14.224999999999998): 66
(14.224999999999998, 17.07): 27
(17.07, 19.915): 13
(19.915, 22.759999999999998): 8
(22.759999999999998, 25.604999999999997): 7
(25.604999999999997, 28.45): 2

Bandra
(0.0, 3.556): 91945
(3.556, 7.112): 1098
(7.112, 10.668): 260
(10.668, 14.224): 122
(14.224, 17.78): 47
(17.78, 21.336): 22
(21.336, 24.892): 6
(24.892, 28.448): 2
(28.448, 32.004): 0
(32.004, 35.56): 2

Byculla
(0.0, 2.769): 91523
(2.769, 5.538): 1262
(5.538, 8.307): 379
(8.307, 11.076): 175
(11.076, 13.845): 96
(13.845, 16.614): 33
(16.614, 19.383000000000003): 16
(19.3830

In [16]:
pred_times = [pd.to_datetime("09:30:00").time(), pd.to_datetime("15:30:00").time(), pd.to_datetime("21:30:00").time(), pd.to_datetime("03:30:00").time()]

In [17]:
# for idx in range(5, aws_df.shape[0]):        # 5 because we are going to remove aws_array[0] and consider 1 hour of observed history
#     date = aws_df.iloc[idx, 0].date()
#     time = aws_df.iloc[idx, 0].time()

#     if(time >= pred_times[0] and time < pred_times[1]):
#         gfs_forecast_filenames = ["".join(str(date).split("-")) + "00" + i for i in [".f003.grib2.nc", ".f006.grib2.nc", ".f009.grib2.nc", ".f012.grib2.nc"]]
#         gfs_analysis_filename = "".join(str(date).split("-")) + "00" + ".f000.grib2.nc"
#     elif(time >= pred_times[1] and time < pred_times[2]):
#         gfs_forecast_filenames = ["".join(str(date).split("-")) + "06" + i for i in [".f003.grib2.nc", ".f006.grib2.nc", ".f009.grib2.nc", ".f012.grib2.nc"]]
#         gfs_analysis_filename = "".join(str(date).split("-")) + "06" + ".f000.grib2.nc"
#     elif(time >= pred_times[2]):
#         gfs_forecast_filenames = ["".join(str(date).split("-")) + "12" + i for i in [".f003.grib2.nc", ".f006.grib2.nc", ".f009.grib2.nc", ".f012.grib2.nc"]]
#         gfs_analysis_filename = "".join(str(date).split("-")) + "12" + ".f000.grib2.nc"
#     elif(time < pred_times[3]):
#         gfs_forecast_filenames = ["".join(str(date - pd.Timedelta(1, unit="D")).split("-")) + "12" + i for i in [".f003.grib2.nc", ".f006.grib2.nc", ".f009.grib2.nc", ".f012.grib2.nc"]]
#         gfs_analysis_filename = "".join(str(date - pd.Timedelta(1, unit="D")).split("-")) + "12" + ".f000.grib2.nc"
#     elif(time >= pred_times[3]):
#         gfs_forecast_filenames = ["".join(str(date - pd.Timedelta(1, unit="D")).split("-")) + "18" + i for i in [".f003.grib2.nc", ".f006.grib2.nc", ".f009.grib2.nc", ".f012.grib2.nc"]]
#         gfs_analysis_filename = "".join(str(date - pd.Timedelta(1, unit="D")).split("-")) + "18" + ".f000.grib2.nc"
    
#     gfs_pr_forecast_pathnames = ["/Users/kpmac/Documents/IITB/DS593/data/GFS/PR/Forecast/full/gfs.0p25." + i for i in gfs_forecast_filenames]
#     gfs_pw_forecast_pathnames = ["/Users/kpmac/Documents/IITB/DS593/data/GFS/PW/Forecast/full/gfs.0p25." + i for i in gfs_forecast_filenames]
#     gfs_rh_forecast_pathnames = ["/Users/kpmac/Documents/IITB/DS593/data/GFS/RH/Forecast/full/gfs.0p25." + i for i in gfs_forecast_filenames]
#     gfs_tcc_forecast_pathnames = ["/Users/kpmac/Documents/IITB/DS593/data/GFS/TCC/Forecast/full/gfs.0p25." + i for i in gfs_forecast_filenames]
    
#     # gfs_pr_analysis_pathname = "/Users/kpmac/Documents/IITB/DS593/data/GFS/PR/Analysis/full/gfs.0p25." + gfs_analysis_filename
#     gfs_pw_analysis_pathname = "/Users/kpmac/Documents/IITB/DS593/data/GFS/PW/Analysis/full/gfs.0p25." + gfs_analysis_filename
#     gfs_rh_analysis_pathname = "/Users/kpmac/Documents/IITB/DS593/data/GFS/RH/Analysis/full/gfs.0p25." + gfs_analysis_filename

#     for i in gfs_pr_forecast_pathnames + gfs_pw_forecast_pathnames + gfs_rh_forecast_pathnames + gfs_tcc_forecast_pathnames:
#         if not os.path.exists(i):
#             print("no", i)
    
#     for i in [gfs_pw_analysis_pathname, gfs_rh_analysis_pathname]:
#         if not os.path.exists(i):
#             print("no", i)

In [18]:
################## Optimize #################### (GFS files will only change after 6 hour intervals, so open the same dataset only once instead of each iteration)

# gfs_features = np.zeros((1, 81*4*4 + 81*2))

# for idx in range(5, aws_df.shape[0]):
#     date = aws_df.iloc[idx, 0].date()
#     time = aws_df.iloc[idx, 0].time()

#     if(time >= pred_times[0] and time < pred_times[1]):
#         gfs_forecast_filenames = ["".join(str(date).split("-")) + "00" + i for i in [".f003.grib2.nc", ".f006.grib2.nc", ".f009.grib2.nc", ".f012.grib2.nc"]]
#         gfs_analysis_filename = "".join(str(date).split("-")) + "00" + ".f000.grib2.nc"
#     elif(time >= pred_times[1] and time < pred_times[2]):
#         gfs_forecast_filenames = ["".join(str(date).split("-")) + "06" + i for i in [".f003.grib2.nc", ".f006.grib2.nc", ".f009.grib2.nc", ".f012.grib2.nc"]]
#         gfs_analysis_filename = "".join(str(date).split("-")) + "06" + ".f000.grib2.nc"
#     elif(time >= pred_times[2]):
#         gfs_forecast_filenames = ["".join(str(date).split("-")) + "12" + i for i in [".f003.grib2.nc", ".f006.grib2.nc", ".f009.grib2.nc", ".f012.grib2.nc"]]
#         gfs_analysis_filename = "".join(str(date).split("-")) + "12" + ".f000.grib2.nc"
#     elif(time < pred_times[3]):
#         gfs_forecast_filenames = ["".join(str(date - pd.Timedelta(1, unit="D")).split("-")) + "12" + i for i in [".f003.grib2.nc", ".f006.grib2.nc", ".f009.grib2.nc", ".f012.grib2.nc"]]
#         gfs_analysis_filename = "".join(str(date - pd.Timedelta(1, unit="D")).split("-")) + "12" + ".f000.grib2.nc"
#     elif(time >= pred_times[3]):
#         gfs_forecast_filenames = ["".join(str(date - pd.Timedelta(1, unit="D")).split("-")) + "18" + i for i in [".f003.grib2.nc", ".f006.grib2.nc", ".f009.grib2.nc", ".f012.grib2.nc"]]
#         gfs_analysis_filename = "".join(str(date - pd.Timedelta(1, unit="D")).split("-")) + "18" + ".f000.grib2.nc"
    
#     gfs_pr_forecast_pathnames = ["/Users/kpmac/Documents/IITB/DS593/data/GFS/PR/Forecast/full/gfs.0p25." + i for i in gfs_forecast_filenames]
#     gfs_pw_forecast_pathnames = ["/Users/kpmac/Documents/IITB/DS593/data/GFS/PW/Forecast/full/gfs.0p25." + i for i in gfs_forecast_filenames]
#     gfs_rh_forecast_pathnames = ["/Users/kpmac/Documents/IITB/DS593/data/GFS/RH/Forecast/full/gfs.0p25." + i for i in gfs_forecast_filenames]
#     gfs_tcc_forecast_pathnames = ["/Users/kpmac/Documents/IITB/DS593/data/GFS/TCC/Forecast/full/gfs.0p25." + i for i in gfs_forecast_filenames]
    
#     # gfs_pr_analysis_pathname = "/Users/kpmac/Documents/IITB/DS593/data/GFS/PR/Analysis/full/gfs.0p25." + gfs_analysis_filename
#     gfs_pw_analysis_pathname = "/Users/kpmac/Documents/IITB/DS593/data/GFS/PW/Analysis/full/gfs.0p25." + gfs_analysis_filename
#     gfs_rh_analysis_pathname = "/Users/kpmac/Documents/IITB/DS593/data/GFS/RH/Analysis/full/gfs.0p25." + gfs_analysis_filename

#     gfs_features_temp = np.array([])
    
#     for path in gfs_pr_forecast_pathnames:
#         file = xr.open_dataset(path)
#         df = file["PRATE_L1_Avg_1"].to_dataframe().reset_index()
#         gfs_features_temp = np.concatenate((gfs_features_temp, df["PRATE_L1_Avg_1"].values), axis=None)

#     for path in gfs_pw_forecast_pathnames:
#         file = xr.open_dataset(path)
#         df = file["P_WAT_L200"].to_dataframe().reset_index()
#         gfs_features_temp = np.concatenate((gfs_features_temp, df["P_WAT_L200"].values), axis=None)

#     for path in gfs_rh_forecast_pathnames:
#         file = xr.open_dataset(path)
#         df = file["R_H_L103"].to_dataframe().reset_index()
#         gfs_features_temp = np.concatenate((gfs_features_temp, df["R_H_L103"].values), axis=None)

#     for path in gfs_tcc_forecast_pathnames:
#         file = xr.open_dataset(path)
#         df = file["T_CDC_L211_Avg_1"].to_dataframe().reset_index()
#         gfs_features_temp = np.concatenate((gfs_features_temp, df["T_CDC_L211_Avg_1"].values), axis=None)

#     file = xr.open_dataset(gfs_pw_analysis_pathname)
#     df = file["P_WAT_L200"].to_dataframe().reset_index()
#     gfs_features_temp = np.concatenate((gfs_features_temp, df["P_WAT_L200"].values), axis=None)

#     file = xr.open_dataset(gfs_rh_analysis_pathname)
#     df = file["R_H_L103"].to_dataframe().reset_index()
#     gfs_features_temp = np.concatenate((gfs_features_temp, df["R_H_L103"].values), axis=None)

#     gfs_features_temp = gfs_features_temp.reshape((1, gfs_features_temp.shape[0]))
#     gfs_features = np.concatenate((gfs_features, gfs_features_temp), axis=0)

In [19]:
# gfs_features_path = "/Users/kpmac/Documents/IITB/DS593/data/preprocessed/gfs_features_array.npy"
# np.save(gfs_features_path, gfs_features)

In [17]:
gfs_features_path = "/Users/kpmac/Documents/IITB/DS593/data/preprocessed/gfs_features_array.npy"
gfs_features = np.load(gfs_features_path)

gfs_features = gfs_features[1:]
print(gfs_features.shape)

(93499, 1458)


In [18]:
aws_df["Time"].dt.year.unique()

array([2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023], dtype=int32)

In [19]:
# Taking 2016-2019 in training data
aws_df[aws_df["Time"].dt.year.isin([2016, 2017, 2018, 2019])].index

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       46838, 46839, 46840, 46841, 46842, 46843, 46844, 46845, 46846, 46847],
      dtype='int64', length=46848)

In [20]:
# Taking 2020-2021 in validation data
aws_df[aws_df["Time"].dt.year.isin([2020, 2021])].index

Index([46848, 46849, 46850, 46851, 46852, 46853, 46854, 46855, 46856, 46857,
       ...
       70262, 70263, 70264, 70265, 70266, 70267, 70268, 70269, 70270, 70271],
      dtype='int64', length=23424)

In [21]:
# Taking 2022-2023 in test data
aws_df[aws_df["Time"].dt.year.isin([2022, 2023])].index

Index([70272, 70273, 70274, 70275, 70276, 70277, 70278, 70279, 70280, 70281,
       ...
       93494, 93495, 93496, 93497, 93498, 93499, 93500, 93501, 93502, 93503],
      dtype='int64', length=23232)

In [22]:
print(aws_array.shape)
print(gfs_features.shape)

(93504, 36)
(93499, 1458)


In [23]:
gfs_train = gfs_features[:46843]      # 46848-5 because we are going to remove aws_array[0] and consider 1 hour of observed history
gfs_val = gfs_features[46843:70267]
gfs_test = gfs_features[70267:]

aws_array_train = aws_array[:46848]

In [24]:
# Standardizing the aws data based on training part
aws_scaler = MinMaxScaler()
aws_scaler.fit(aws_array_train)

aws_standardized = aws_scaler.transform(aws_array)

In [25]:
aws_standardized.shape

(93504, 36)

In [26]:
aws_features = []
y = []

for idx in range(5, aws_standardized.shape[0]):        # 5 because we are going to remove aws_array[0] and consider 1 hour of observed history
    aws_features.append(aws_standardized[idx-4:idx])   # past one hour
    y.append(aws_standardized[idx])

In [27]:
aws_features = np.array(aws_features)
y = np.array(y)

In [28]:
print(aws_features.shape)
print(y.shape)

(93499, 4, 36)
(93499, 36)


In [29]:
aws_train = aws_features[:46843]    # 46848-5 because we are going to remove aws_array[0] and consider 1 hour of observed history
aws_val = aws_features[46843:70267]
aws_test = aws_features[70267:]

In [30]:
y_train = y[:46843]    # 46848-5 because we are going to remove aws_array[0] and consider 1 hour of observed history
y_val = y[46843:70267]
y_test = y[70267:]

In [31]:
# Standardizing the gfs features based on training part
gfs_scaler = MinMaxScaler()
gfs_scaler.fit(gfs_train)

gfs_train = gfs_scaler.transform(gfs_train)
gfs_val = gfs_scaler.transform(gfs_val)
gfs_test = gfs_scaler.transform(gfs_test)

In [32]:
print(aws_train.shape)
print(gfs_train.shape)

(46843, 4, 36)
(46843, 1458)


In [34]:
%%time
# Reducing dimensionality of gfs_features from 1458 to n principal components contributing to 90% variance
pca = PCA(0.9)
pca.fit(gfs_train)        # get tranformation map from training data
print(pca.n_components_)

gfs_train_reduced = pca.transform(gfs_train)
gfs_val_reduced = pca.transform(gfs_val)
gfs_test_reduced = pca.transform(gfs_test)

32
CPU times: user 44.7 s, sys: 3.08 s, total: 47.8 s
Wall time: 28.4 s


In [35]:
print(gfs_train_reduced.shape)

(46843, 32)


In [104]:
# %%time
# np.concatenate((X[:, :, 0], gfs_features_normalized), axis=1).shape

CPU times: user 8.29 s, sys: 7.74 s, total: 16 s
Wall time: 16.3 s


(93499, 1462)

In [105]:
# %%time
# np.append(X[:, :, 0], gfs_features_normalized, axis=1).shape

CPU times: user 8.06 s, sys: 8.21 s, total: 16.3 s
Wall time: 17.6 s


(93499, 1462)

In [36]:
y_train.shape

(46843, 36)

In [37]:
lin_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = LinearRegression().fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    lin_reg_list.append(reg)

Andheri training score: 0.41152566691120107 validation score: 0.4347087981414317
B ward training score: 0.40314127243359854 validation score: 0.5022416324033716
Bandra training score: 0.41732792907884964 validation score: 0.4741932004211249
Byculla training score: 0.3879058487156294 validation score: 0.5134037518206107
C ward training score: 0.38445528260624295 validation score: 0.5123139955007963
Chembur training score: 0.3761361468698051 validation score: 0.457152215269935
Chincholi training score: 0.42905905330812577 validation score: 0.46487843923527483
Colaba training score: 0.4034061680081563 validation score: 0.49810835462054703
D Ward training score: 0.3733717867382622 validation score: 0.474379231704524
Dahisar training score: 0.43335999227311517 validation score: 0.4575127960597488
Dindoshi training score: 0.4214397406263416 validation score: 0.4524241156828591
F North training score: 0.4096942974252509 validation score: 0.4680941504841687
F South training score: 0.3978542065

In [39]:
for i in range(len(lin_reg_list)):
    X_test = np.concatenate((aws_test[:, :, i], gfs_test_reduced), axis=1)
    print(regions[i], "Test R2-score:", reg.score(X_test, y_test[:, i]))

Andheri Test R2-score: 0.34037245391983617
B ward Test R2-score: 0.31262233782613646
Bandra Test R2-score: 0.3200387696456767
Byculla Test R2-score: 0.32811606378566327
C ward Test R2-score: 0.3053474503876432
Chembur Test R2-score: 0.31410293969179026
Chincholi Test R2-score: 0.3045634100136856
Colaba Test R2-score: 0.31098260594692817
D Ward Test R2-score: 0.26155577208370673
Dahisar Test R2-score: 0.3666454083909243
Dindoshi Test R2-score: 0.31867780963555203
F North Test R2-score: 0.3267544795993418
F South Test R2-score: 0.29389090853826105
G South Test R2-score: 0.2988581937899367
Gowanpada Test R2-score: 0.24344197291140957
H West ward Test R2-score: 0.34895473969624224
K East ward Test R2-score: 0.3240759444789555
K West ward Test R2-score: 0.3124699184832427
Kandivali Test R2-score: 0.3517223687933104
Kurla Test R2-score: 0.3059964779898908
L ward Test R2-score: 0.31985789820142885
M West ward Test R2-score: 0.34011576054996706
MCGM 1 Test R2-score: 0.28872374122560185
Malvani

In [41]:
val_score_list = []
for i in range(len(lin_reg_list)):
    reg = lin_reg_list[i]
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    val_score_list.append(reg.score(X_val, y_val[:, i]))
val_score_list = np.array(val_score_list)
print(val_score_list.mean())

0.46736964318116603


In [86]:
lin_reg_list[0].predict(np.concatenate((aws_val[:, :, 0], gfs_val_reduced), axis=1)).max()

0.9295770880380139

In [98]:
ridge_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = Ridge(alpha=0.5).fit(X_train, y_train[:, i])        # increasing alpha makes score worse
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    ridge_reg_list.append(reg)

Andheri training score: 0.4115008539783118 validation score: 0.4343930328660798
B ward training score: 0.4031074073614985 validation score: 0.5017273817970789
Bandra training score: 0.4173019575494423 validation score: 0.4738909199132595
Byculla training score: 0.3878763179308299 validation score: 0.5125920031545392
C ward training score: 0.3844248214920434 validation score: 0.511478004405016
Chembur training score: 0.37610684573526154 validation score: 0.45643753323980907
Chincholi training score: 0.429013171974029 validation score: 0.46410663890069714
Colaba training score: 0.40337543927250263 validation score: 0.49767253503337716
D Ward training score: 0.3733281016169292 validation score: 0.4740036571955214
Dahisar training score: 0.4333376072998544 validation score: 0.45708737152696155
Dindoshi training score: 0.42140534058568413 validation score: 0.45185800073722693
F North training score: 0.409632447096111 validation score: 0.46751625341843994
F South training score: 0.3978371287

In [89]:
elastic_net_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = ElasticNet(alpha=0.00001, l1_ratio=0.001).fit(X_train, y_train[:, i])        # smaller alpha and l1_ratio gives better results (large giving 0 or negative score)
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    elastic_net_reg_list.append(reg)

Andheri training score: 0.411503765390528 validation score: 0.4344130295579629
B ward training score: 0.40311136867257236 validation score: 0.50176035224671
Bandra training score: 0.4173050036514927 validation score: 0.4739111122436086
Byculla training score: 0.38787977093798887 validation score: 0.5126431664424953
C ward training score: 0.3844283883210451 validation score: 0.5115309397647947
Chembur training score: 0.37611038552169196 validation score: 0.4564832032698475
Chincholi training score: 0.4290186955453149 validation score: 0.46415707626309477
Colaba training score: 0.40337903696252797 validation score: 0.4977021953457448
D Ward training score: 0.3733333638697529 validation score: 0.47402763363011036
Dahisar training score: 0.4333403208375598 validation score: 0.4571154715450789
Dindoshi training score: 0.4214094910155093 validation score: 0.4518951793697972
F North training score: 0.40963964240568174 validation score: 0.4675548695425331
F South training score: 0.397839134060

In [90]:
lars_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = Lars().fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    lars_reg_list.append(reg)

Andheri training score: 0.4115256669112012 validation score: 0.4347087981414285
B ward training score: 0.40314127243359865 validation score: 0.5022416324033572
Bandra training score: 0.41732792907884964 validation score: 0.4741932004211359
Byculla training score: 0.3879058487156294 validation score: 0.5134037518206209
C ward training score: 0.38445528260624307 validation score: 0.5123139955008168
Chembur training score: 0.3761361465239048 validation score: 0.4571507892443756
Chincholi training score: 0.42905905330812577 validation score: 0.4648784392352663
Colaba training score: 0.4034061680081562 validation score: 0.4981083546205435
D Ward training score: 0.3733717867382622 validation score: 0.47437923170450547
Dahisar training score: 0.43335999227311517 validation score: 0.45751279605975614
Dindoshi training score: 0.4214397406263417 validation score: 0.45242411568286156
F North training score: 0.4096942974252509 validation score: 0.4680941504841736
F South training score: 0.39785420

In [102]:
lasso_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = Lasso(alpha=0.00005).fit(X_train, y_train[:, i])        # reducing alpha makes score worse (0 or negative)
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    lasso_reg_list.append(reg)

Andheri training score: 0.40975404073151644 validation score: 0.4310271606999031
B ward training score: 0.4004702444923278 validation score: 0.49256728449486986
Bandra training score: 0.4150719241086269 validation score: 0.4695148787583068
Byculla training score: 0.38543372359635264 validation score: 0.5025265857219106
C ward training score: 0.382047938024759 validation score: 0.5016878004396084
Chembur training score: 0.37362354242375895 validation score: 0.4493556635773651
Chincholi training score: 0.42582505612358623 validation score: 0.4593186619729812
Colaba training score: 0.40120945022945387 validation score: 0.49152949873501994
D Ward training score: 0.36979673100527566 validation score: 0.45972968288570293
Dahisar training score: 0.4317653322818126 validation score: 0.45556624400739054
Dindoshi training score: 0.41889562781255474 validation score: 0.4470482204127494
F North training score: 0.4053656599420101 validation score: 0.45885917314711866
F South training score: 0.39650

In [107]:
lasso_lars_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = LassoLars(alpha=0.00005).fit(X_train, y_train[:, i])        # reducing alpha makes score worse (0 or negative)
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    lasso_lars_reg_list.append(reg)

Andheri training score: 0.4097537359250355 validation score: 0.4310270785178315
B ward training score: 0.40046960601141857 validation score: 0.492565928706049
Bandra training score: 0.4150716234159544 validation score: 0.4695144247976045
Byculla training score: 0.3854332471575127 validation score: 0.5025255565633835
C ward training score: 0.3820473902602449 validation score: 0.501687142294857
Chembur training score: 0.37362281249672424 validation score: 0.4493539476405173
Chincholi training score: 0.4258241949619136 validation score: 0.45931798627314824
Colaba training score: 0.4012089659706055 validation score: 0.49152906165158194
D Ward training score: 0.3697963057149607 validation score: 0.4597287163158098
Dahisar training score: 0.43176476825609766 validation score: 0.4555669323170036
Dindoshi training score: 0.4188951768833271 validation score: 0.44704782821311384
F North training score: 0.4053649387316498 validation score: 0.45885855312161195
F South training score: 0.39650021937

In [116]:
orthogonal_matching_pursuit_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = OrthogonalMatchingPursuit(n_nonzero_coefs=X_train.shape[1]).fit(X_train, y_train[:, i])        # reducing n_nonzer_coefs makes scores worse
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    orthogonal_matching_pursuit_reg_list.append(reg)

Andheri training score: 0.41152566691120107 validation score: 0.4347087981414284
B ward training score: 0.40314127243359865 validation score: 0.5022416324033572
Bandra training score: 0.41732792907884964 validation score: 0.4741932004211359
Byculla training score: 0.3879058487156294 validation score: 0.5134037518206209
C ward training score: 0.38445528260624307 validation score: 0.5123139955008168
Chembur training score: 0.376136146869805 validation score: 0.45715221526994465
Chincholi training score: 0.42905905330812577 validation score: 0.4648784392352663
Colaba training score: 0.4034061680081562 validation score: 0.4981083546205435
D Ward training score: 0.3733717867382622 validation score: 0.47437923170450547
Dahisar training score: 0.43335999227311517 validation score: 0.45751279605975614
Dindoshi training score: 0.4214397406263417 validation score: 0.45242411568286156
F North training score: 0.4096942974252509 validation score: 0.46809415048417347
F South training score: 0.397854

In [117]:
ard_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = ARDRegression().fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    ard_reg_list.append(reg)

Andheri training score: 0.4050304402185909 validation score: 0.4303821739138628
B ward training score: 0.39693614468033944 validation score: 0.4994522187884993
Bandra training score: 0.4110669924648933 validation score: 0.47072033101315436
Byculla training score: 0.381103046725046 validation score: 0.5117187410761896
C ward training score: 0.3779885388191009 validation score: 0.5094840094031987
Chembur training score: 0.36849498802429426 validation score: 0.45219626989143846
Chincholi training score: 0.4225473388866503 validation score: 0.4620632274295132
Colaba training score: 0.39769555366837206 validation score: 0.49469466960018205
D Ward training score: 0.36694204645933126 validation score: 0.47190464242167796
Dahisar training score: 0.42706077758988437 validation score: 0.45445838235669345
Dindoshi training score: 0.41431396214923655 validation score: 0.4515160612420228
F North training score: 0.4024030893452184 validation score: 0.46400798022687517
F South training score: 0.39012

In [118]:
bayesian_ridge_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = BayesianRidge().fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    bayesian_ridge_reg_list.append(reg)

Andheri training score: 0.41152463445746457 validation score: 0.43464812362061656
B ward training score: 0.4031402020564714 validation score: 0.5021548671246243
Bandra training score: 0.41732691534576294 validation score: 0.47413754808602204
Byculla training score: 0.38790468679378365 validation score: 0.5132471634943703
C ward training score: 0.38445414239362885 validation score: 0.5121566951831176
Chembur training score: 0.3761349353148694 validation score: 0.4570113473639149
Chincholi training score: 0.4290580621196133 validation score: 0.4647703570767202
Colaba training score: 0.40340510830506604 validation score: 0.4980321903424889
D Ward training score: 0.3733706114521482 validation score: 0.47432383109880316
Dahisar training score: 0.43335906220375264 validation score: 0.4574296399501804
Dindoshi training score: 0.4214387015420362 validation score: 0.4523308040288867
F North training score: 0.4096932516809296 validation score: 0.46802572044209334
F South training score: 0.397853

In [90]:
linear_svr_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = LinearSVR(epsilon=0.0).fit(X_train, y_train[:, i])        # larger values of epsilon give worse scores
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    linear_svr_list.append(reg)

Andheri training score: 0.3237137261207692 validation score: 0.3177666434161547


/Users/kpmac/Documents/IITB/DS593/itu/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
nu_svr_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = NuSVR(nu=0.5, C=1.0).fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    nu_svr_list.append(reg)

Andheri training score: 0.3868924596805764 validation score: 0.272576040861782
B ward training score: 0.36096219236651284 validation score: 0.3011737892016869


In [ ]:
svr_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = SVR().fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    svr_list.append(reg)

Andheri training score: -4.062677100010622 validation score: -2.8596140991818304
B ward training score: -4.499332291149026 validation score: -2.954740422027807
Bandra training score: -3.7879199456976904 validation score: -2.1685167548592914
Byculla training score: -4.6263930771776165 validation score: -3.2960492426767756


In [40]:
dec_tree_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = DecisionTreeRegressor().fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    dec_tree_reg_list.append(reg)

Andheri training score: 0.919906735489113 validation score: -0.40343489609057426
B ward training score: 0.9203587184011333 validation score: -0.3166529139119467
Bandra training score: 0.9323529408239783 validation score: -0.3808362500189677
Byculla training score: 0.9328011125217703 validation score: -0.26874806587590694
C ward training score: 0.9009869737526303 validation score: -0.18635287043366544
Chembur training score: 0.9228647244917716 validation score: -0.4319979038297719
Chincholi training score: 0.9351902095139826 validation score: -0.40554413023949465
Colaba training score: 0.9271384817498876 validation score: -0.6790604605499102
D Ward training score: 0.929341088434784 validation score: -0.3196953539114793
Dahisar training score: 0.9463510932592214 validation score: -0.3247326928441694
Dindoshi training score: 0.930272340036413 validation score: -0.4466105368444029
F North training score: 0.9343211115920363 validation score: -0.3830879229643125
F South training score: 0.907

In [52]:
dec_tree_reg_list1 = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = DecisionTreeRegressor(max_depth=3).fit(X_train, y_train[:, i])        # decreasing max_depth causes underfitting, increasing causes overfitting
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    dec_tree_reg_list1.append(reg)
    break

Andheri training score: 0.410093844157413 validation score: 0.4167882311192108


In [61]:
dec_tree_reg_list2 = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = DecisionTreeRegressor(min_weight_fraction_leaf=0.001).fit(X_train, y_train[:, i])        # decreasing min_weight_fraction_leaf causes overfitting, increasing causes underfitting
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    dec_tree_reg_list2.append(reg)
    break

Andheri training score: 0.45977339717038235 validation score: 0.3908854628924152


In [71]:
dec_tree_reg_list2 = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = DecisionTreeRegressor(max_leaf_nodes=10).fit(X_train, y_train[:, i])        # decreasing max_leaf_nodes causes underfitting, increasing causes overfitting
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    dec_tree_reg_list2.append(reg)
    break

Andheri training score: 0.4269356327319206 validation score: 0.3990331896786694


In [76]:
extra_tree_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = ExtraTreeRegressor().fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    extra_tree_reg_list.append(reg)

Andheri training score: 0.919906735489113 validation score: -0.39404690674828635
B ward training score: 0.9203587184011333 validation score: -0.0923725759691183
Bandra training score: 0.9323529408239783 validation score: -0.34605900336359774
Byculla training score: 0.9328011125217703 validation score: -0.19219764458013833
C ward training score: 0.9009869737526303 validation score: -0.1519992232103804
Chembur training score: 0.9228647244917716 validation score: -0.4673020882398844
Chincholi training score: 0.9351902095139826 validation score: -0.231184266984501
Colaba training score: 0.9271384817498876 validation score: -0.25797100507140547
D Ward training score: 0.929341088434784 validation score: -0.11457981235065473
Dahisar training score: 0.9463510932592214 validation score: -0.47055517729395846
Dindoshi training score: 0.930272340036413 validation score: -0.4584339630057701
F North training score: 0.9343211115920363 validation score: -0.4125406687249398
F South training score: 0.90

In [85]:
ada_boost_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    tree = DecisionTreeRegressor(max_depth=50)
    reg = AdaBoostRegressor(estimator=tree, n_estimators=25).fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    ada_boost_reg_list.append(reg)
    break

Andheri training score: 0.8217064649644963 validation score: 0.3100398628720257


In [86]:
bag_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    tree = DecisionTreeRegressor(max_depth=50)
    reg = BaggingRegressor(estimator=tree, n_estimators=25).fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    bag_reg_list.append(reg)
    break

Andheri training score: 0.8291775834558314 validation score: 0.3005830676417457


In [99]:
extra_trees_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = ExtraTreesRegressor(n_estimators=100).fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    extra_trees_reg_list.append(reg)
    break

Andheri training score: 0.9199067354891131 validation score: 0.36525538620648


In [38]:
gradient_boosting_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = GradientBoostingRegressor(n_estimators=50).fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    gradient_boosting_reg_list.append(reg)

Andheri training score: 0.4882605572919275 validation score: 0.4023608513667417
B ward training score: 0.4806432458465114 validation score: 0.4417223682974645
Bandra training score: 0.5039288721070243 validation score: 0.4328328827889266
Byculla training score: 0.46125261884882884 validation score: 0.48871719412797543
C ward training score: 0.45657379377721674 validation score: 0.48211243039712826
Chembur training score: 0.4459215913560838 validation score: 0.4490929149106372
Chincholi training score: 0.5009618842268373 validation score: 0.45719888030625766
Colaba training score: 0.4836250501827736 validation score: 0.4686529991213779
D Ward training score: 0.4675662539458064 validation score: 0.4149585164192937
Dahisar training score: 0.5026343800601502 validation score: 0.4530330063951551
Dindoshi training score: 0.4835213414259535 validation score: 0.4483713520353265
F North training score: 0.4782500973192092 validation score: 0.4559657054806412
F South training score: 0.46587604313

In [92]:
hist_gradient_boosting_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = HistGradientBoostingRegressor().fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    hist_gradient_boosting_reg_list.append(reg)
    break

Andheri training score: 0.5737219156702704 validation score: 0.3883717955088878


In [96]:
random_forest_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    reg = RandomForestRegressor(n_estimators=50).fit(X_train, y_train[:, i])
    print(regions[i], "training score:", reg.score(X_train, y_train[:, i]), "validation score:", reg.score(X_val, y_val[:, i]))
    random_forest_reg_list.append(reg)
    break

Andheri training score: 0.8375927619185212 validation score: 0.312321841898084


In [50]:
xgb_reg_list = []
for i in range(y.shape[1]):
    X_train = np.concatenate((aws_train[:, :, i], gfs_train_reduced), axis=1)
    X_val = np.concatenate((aws_val[:, :, i], gfs_val_reduced), axis=1)
    dtrain_reg = xgb.DMatrix(X_train, y_train[:, i])
    dval_reg = xgb.DMatrix(X_val, y_val[:, i])
    params = {
        "objective": "reg:squarederror",
        "tree_method": "hist",
    }
    n = 100
    evals = [(dtrain_reg, "train"), (dval_reg, "validation")]
    model = xgb.train(
        params=params,
        dtrain=dtrain_reg,
        num_boost_round=n,
        evals=evals,
        early_stopping_rounds=3,
    )
    xgb_train_preds = model.predict(dtrain_reg)
    xgb_val_preds = model.predict(dval_reg)
    print(regions[i], "Training score:", r2_score(y_train[:, i], xgb_train_preds), "Validation score:", r2_score(y_val[:, i], xgb_val_preds))
    print()
    xgb_reg_list.append(model)

[0]	train-rmse:0.03481	validation-rmse:0.04193
[1]	train-rmse:0.03157	validation-rmse:0.03959
[2]	train-rmse:0.02947	validation-rmse:0.03826
[3]	train-rmse:0.02816	validation-rmse:0.03739
[4]	train-rmse:0.02723	validation-rmse:0.03713
[5]	train-rmse:0.02656	validation-rmse:0.03680
[6]	train-rmse:0.02596	validation-rmse:0.03680
[7]	train-rmse:0.02556	validation-rmse:0.03685
[8]	train-rmse:0.02520	validation-rmse:0.03683
[9]	train-rmse:0.02489	validation-rmse:0.03684
Andheri Training score: 0.6126548777116758 Validation score: 0.36264176192010644

[0]	train-rmse:0.03293	validation-rmse:0.04178
[1]	train-rmse:0.02984	validation-rmse:0.03940
[2]	train-rmse:0.02785	validation-rmse:0.03765
[3]	train-rmse:0.02655	validation-rmse:0.03690
[4]	train-rmse:0.02567	validation-rmse:0.03641
[5]	train-rmse:0.02503	validation-rmse:0.03635
[6]	train-rmse:0.02446	validation-rmse:0.03644
[7]	train-rmse:0.02398	validation-rmse:0.03649
[8]	train-rmse:0.02361	validation-rmse:0.03641
B ward Training score: 0.